In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tqdm

import resources

In [2]:
exp = resources.load('Test 5')

In [3]:
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))): # if value ist tensor
        value = value.numpy() # get value of tensor
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a floast_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def serialize_array(array):
  array = tf.io.serialize_tensor(array)
  return array

In [4]:
def parse_ae_data(ae, file_id):
    data = {
        'file_id': _int64_feature(file_id),
        'fs': _int64_feature(2_000_000),
        'ae': _bytes_feature(serialize_array(ae)),
    }

    out = tf.train.Example(features=tf.train.Features(feature=data))
    return out

In [5]:
opt = tf.io.TFRecordOptions(compression_type='GZIP')
with tf.io.TFRecordWriter('test_5_ae_comp.tfrecord', opt) as f:
    for file_id in tqdm.tqdm(range(20)):
        ae = exp.ae.readAE(file_id)
        out = parse_ae_data(ae, file_id)
        f.write(out.SerializeToString())

100%|██████████| 20/20 [04:21<00:00, 13.09s/it]


In [6]:
with tf.io.TFRecordWriter('test_5_ae.tfrecord') as f:
    for file_id in tqdm.tqdm(range(20)):
        ae = exp.ae.readAE(file_id)
        out = parse_ae_data(ae, file_id)
        f.write(out.SerializeToString())

100%|██████████| 20/20 [01:00<00:00,  3.02s/it]


In [2]:
def parse_tfr_ae(element):
    data = {
        'file_id': tf.io.FixedLenFeature([], tf.int64),
        'fs': tf.io.FixedLenFeature([], tf.int64),
        'ae': tf.io.FixedLenFeature([], tf.string),
    }

    sample = tf.io.parse_single_example(element, data)
    return sample

In [3]:
dataset = tf.data.TFRecordDataset('test_5_ae.tfrecord')
dataset = dataset.map(parse_tfr_ae)
dataset

<_MapDataset element_spec={'ae': TensorSpec(shape=(), dtype=tf.string, name=None), 'file_id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'fs': TensorSpec(shape=(), dtype=tf.int64, name=None)}>

In [4]:
for sample in dataset:
    ae_sig = sample['ae']
    ae_sig = tf.io.parse_tensor(ae_sig, out_type=tf.double)
    print(f'File {sample["file_id"].numpy()}: {ae_sig.numpy().shape} ')

File 0: (42326800,) 
File 1: (41863488,) 
File 2: (41912320,) 
File 3: (42094416,) 
File 4: (41818496,) 
File 5: (42322272,) 
File 6: (42407440,) 
File 7: (42369840,) 
File 8: (42026576,) 
File 9: (41685136,) 
File 10: (42336048,) 
File 11: (42352672,) 
File 12: (41765056,) 
File 13: (41701408,) 
File 14: (41503568,) 
File 15: (41914976,) 
File 16: (42218192,) 
File 17: (42260160,) 
File 18: (41559216,) 
File 19: (41885680,) 


In [6]:
for sample in dataset:
    sample['file_id'].numpy()